<a href="https://colab.research.google.com/github/yeonghun00/stock-notes/blob/main/analysis/leveragne%20and%20factors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import zipfile
import pandas as pd
import numpy as np
import io
import plotly.graph_objects as go
from scipy.stats import gmean

In [2]:
def get_df(url):
  df = pd.read_csv(url)
  df['Date'] = pd.to_datetime(df['Date'])
  df.index = df['Date']
  df.drop(columns=['Date'], inplace=True)
  return df


In [3]:
market = get_df('https://stooq.com/q/d/l/?s=^spx&i=d')

In [4]:
market

,Open,High,Low,Close,Volume
Date,,,,,
1789-05-01,0.51,0.51,0.51,0.51,NaN
1789-06-01,0.51,0.51,0.51,0.51,NaN
1789-07-01,0.50,0.50,0.50,0.50,NaN
1789-08-01,0.50,0.51,0.50,0.51,NaN
1789-09-01,0.51,0.51,0.50,0.51,NaN
...,...,...,...,...,...
2024-04-22,4987.33,5038.84,4969.40,5010.60,2.352809e+09
2024-04-23,5028.85,5076.12,5027.96,5070.55,2.346600e+09
2024-04-24,5084.86,5089.48,5047.02,5071.63,2.548619e+09


In [13]:
market['Close_120'] = market['Close'].rolling(window=120).mean()
date_range = pd.date_range(start=market.index.min(), end=market.index.max(), freq='D')
market = pd.DataFrame(data=market, index=date_range).bfill()

In [14]:
gdp_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1318&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=GDP&scale=left&cosd=1947-01-01&coed=2024-01-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Quarterly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-04-27&revision_date=2024-04-27&nd=1947-01-01'
interest_rate_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1318&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=FEDFUNDS&scale=left&cosd=1954-07-01&coed=2024-03-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-04-27&revision_date=2024-04-27&nd=1954-07-01'
unemployment_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1318&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=UNRATE&scale=left&cosd=1948-01-01&coed=2024-03-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-04-27&revision_date=2024-04-27&nd=1948-01-01'
m2_url = 'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1318&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=M2SL&scale=left&cosd=1959-01-01&coed=2024-03-01&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=2&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date=2024-04-28&revision_date=2024-04-28&nd=1959-01-01'

gdp = pd.read_csv(gdp_url)
interest_rate = pd.read_csv(interest_rate_url)
unemployment = pd.read_csv(unemployment_url)
m2 = pd.read_csv(m2_url)

gdp.index = pd.to_datetime(gdp['DATE'])
interest_rate.index = pd.to_datetime(interest_rate['DATE'])
unemployment.index = pd.to_datetime(unemployment['DATE'])
m2.index = pd.to_datetime(m2['DATE'])

In [15]:
market['Close_120']

1789-05-01       0.507500
1789-05-02       0.507500
1789-05-03       0.507500
1789-05-04       0.507500
1789-05-05       0.507500
                 ...     
2024-04-22    5004.839417
2024-04-23    5007.304417
2024-04-24    5009.778417
2024-04-25    5012.002083
2024-04-26    5014.640500
Freq: D, Name: Close_120, Length: 85828, dtype: float64

In [16]:
df = pd.DataFrame({
    'market': market['Close_120'],
    'gdp': gdp['GDP'],
    'interest_rate': interest_rate['FEDFUNDS'],
    'unemployment': unemployment['UNRATE'],
    'm2': m2['M2SL']
})

In [17]:
df = df[df.index.is_quarter_start].bfill().dropna()

In [19]:
df['market_change'] = df['market'].pct_change()

In [22]:
df.index[df.index > '1960-01-01']

DatetimeIndex(['1960-04-01', '1960-07-01', '1960-10-01', '1961-01-01',
               '1961-04-01', '1961-07-01', '1961-10-01', '1962-01-01',
               '1962-04-01', '1962-07-01',
               ...
               '2021-10-01', '2022-01-01', '2022-04-01', '2022-07-01',
               '2022-10-01', '2023-01-01', '2023-04-01', '2023-07-01',
               '2023-10-01', '2024-01-01'],
              dtype='datetime64[ns]', length=256, freq=None)

In [39]:
(df['gdp'] / df['m2'])

1789-07-01    0.848444
1789-10-01    0.848444
1790-01-01    0.848444
1790-04-01    0.848444
1790-07-01    0.848444
                ...   
2023-01-01    1.263434
2023-04-01    1.304562
2023-07-01    1.327844
2023-10-01    1.351121
2024-01-01    1.362832
Length: 939, dtype: float64

1950-04-01      50.066034
1950-07-01      61.630600
1950-10-01      76.177381
1951-01-01      90.810811
1951-04-01     110.996774
                 ...     
2023-01-01    7886.353235
2023-04-01    7959.709412
2023-07-01    7888.608000
2023-10-01    7357.104737
2024-01-01    7644.458919
Length: 296, dtype: float64

In [56]:
# prompt: plot market / gdp ratio with other line that is expoenthial function that best fit the data

import plotly.graph_objects as go

start_date = '1950-01-01'
t = (df['m2'] / df['interest_rate'])
t = t[t.index > start_date]

# Create the plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=df[df.index > start_date].index, y=t, name='Market/GDP Ratio'))
# fig.add_trace(go.Scatter(x=df.index[df.index > '1960-01-01'], y=df['m2'][df.index>'1960-01-01']))

fig.update_layout(title='Market/GDP Ratio with Exponential Fit', xaxis_title='Date', yaxis_title='Market/GDP Ratio')

fig.show()


In [54]:
# prompt: plot market / gdp ratio with other line that is expoenthial function that best fit the data

import plotly.graph_objects as go

start_date = '1950-01-01'
t = (df['market'])
t = t[t.index > start_date]

# Create the plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=df[df.index > start_date].index, y=t, name='Market/GDP Ratio'))

fig.update_layout(title='Market/GDP Ratio with Exponential Fit', xaxis_title='Date', yaxis_title='Market/GDP Ratio')

fig.show()


In [57]:
# prompt: plot market / gdp ratio with other line that is expoenthial function that best fit the data

import plotly.graph_objects as go

df['market_gdp_ratio'] = df['market'] / df['gdp']

# Fit an exponential function to the market_gdp_ratio data
params = np.polyfit(df.index.to_julian_date(), np.log(df['market_gdp_ratio']), 1)

# Create a new dataframe with the fitted exponential function
df_fit = pd.DataFrame({
    'index': df.index,
    'market_gdp_ratio_fit': np.exp(params[1]) * np.exp(params[0] * df.index.to_julian_date())
})

# Create the plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df['market_gdp_ratio'], name='Market/GDP Ratio'))
fig.add_trace(go.Scatter(x=df_fit['index'], y=df_fit['market_gdp_ratio_fit'], name='Exponential Fit'))

fig.update_layout(title='Market/GDP Ratio with Exponential Fit', xaxis_title='Date', yaxis_title='Market/GDP Ratio')

fig.show()


In [53]:
df

,market,gdp,interest_rate,unemployment
1789-07-01,0.625917,243.164,0.80,3.4
1789-10-01,0.625917,243.164,0.80,3.4
1790-01-01,0.625917,243.164,0.80,3.4
1790-04-01,0.625917,243.164,0.80,3.4
1790-07-01,0.625917,243.164,0.80,3.4
...,...,...,...,...
2023-01-01,3920.517750,26813.601,4.33,3.4
2023-04-01,3937.946917,27063.012,4.83,3.4
2023-07-01,4116.364417,27610.128,5.12,3.5
2023-10-01,4344.075083,27956.998,5.33,3.8
